In [2]:
import torch
import random

def make_masks(num_patchs, mC_x=12, p_n_y=0.5, p_c_y=0.2):
    
    C, N = num_patchs
    
    while True:
        mask_x = []# mN, mC
        mask_y = []
        mask_y_bx = []
        for i in range(N):
            c_idx = torch.randperm(C) + i*C
            if random.random()>p_n_y:
                mask_x.append(c_idx[:mC_x])
                mask_y_bx.append(c_idx[mC_x:])
            else:
                mask_y.append(c_idx)
        if len(mask_x)==0: continue
        if len(mask_y_bx)==0: continue
        mask_y_bx = torch.cat(mask_y_bx, dim=0)
        mask_y_bx = mask_y_bx[torch.rand(mask_y_bx.shape)<p_c_y]
        if len(mask_y_bx)==0: continue
        break
    
    return torch.stack(mask_x, dim=0), torch.cat(mask_y+[mask_y_bx], dim=0)
mask_x, mask_y = make_masks(((62), (80)))
print('mask_x: ', mask_x.shape)
print('mask_y: ', mask_y.shape)

mask_x:  torch.Size([32, 12])
mask_y:  torch.Size([3293])


In [ ]:
def apply_mask(mask, x): # k, (32, 80, 62, 128)
    """
    :param x: tensor of shape [B (batch-size), N (num-patches), C, D (feature-dim)]
    :param mask: tensor [mN, mC] containing indices of patches in [N, C] to keep 
    """    
    B, N, C, D = x.shape # 32, 80, 62, 128
    if len(mask.shape)==2:
        mN, mC = mask.shape
        
        mask_keep = mask.reshape((1,mN*mC,1)).repeat((B, 1, D))
        masked_x = torch.gather(x.reshape((B, N*C, D)), dim=-2, index=mask_keep)
        masked_x = masked_x.contiguous().view((B,mN,mC,D))
    else:
        mN = mask.shape[0]
        
        mask_keep = mask.reshape((1,mN,1)).repeat((B, 1, D))
        masked_x = torch.gather(x.reshape((B, N*C, D)), dim=-2, index=mask_keep)
    return masked_x

x = torch.rand(32, 80, 62, 128)
x = apply_mask(mask_x, x)
print(x.shape)

torch.Size([32, 32, 12, 128])


In [7]:
y = torch.rand(32, 62, 80, 41)
y = apply_mask(mask_y, y)
print(y.shape)

torch.Size([32, 3293, 41])


In [8]:
emb_text = torch.rand(32, 56, 1024)
# self.patch_embed = patch_module(
#     img_size=img_size,# C, T [56, 1024] 
#     patch_size=patch_size, # 32
#     patch_stride=patch_stride, # 32
#     embed_dim=embed_dim) # 128
# self.num_patches = self.patch_embed.num_patches# (56, 32)

mask_x, mask_y = make_masks(((56), (32)))
print(mask_x.shape)
print(mask_y.shape)

torch.Size([15, 12])
torch.Size([1070])


In [10]:
C, N = ((56), (32))
y = emb_text
block_size_c, block_size_n = y.shape[-2]//C, y.shape[-1]//N # 41,1 
y = y.view(y.shape[0], C, block_size_c, N, block_size_n)
y = y.permute(0, 3, 1, 2, 4).contiguous() # B, N, C, bc, bn # [32, 80, 62, 1, 41
y = y.view(y.shape[0], C, N, block_size_c * block_size_n) # [32, 62, 80, 41]
y = apply_mask(mask_y.to(y.device), y)
print(y.shape)

torch.Size([32, 1070, 32])
